<a href="https://colab.research.google.com/github/Melisa7L/PPS/blob/main/Algoritmo_kmeans_PPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En base al codigo de la nube de la palabras realice modificaciones para que con las palabras frecuentes que se encontraron se pueda implementar el algoritmo kmeans generando clusters.

Librerias que se necesitan:

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
!pip install spacy

In [18]:
!pip install click

In [19]:
!pip install --upgrade click

In [20]:
!python -m spacy download es_core_news_sm

2023-11-17 02:01:31.074855: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 02:01:31.075024: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 02:01:31.075231: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 02:01:33.843453: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 19.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


Se construyó la matriz de co-ocurrencia entre palabras en un contexto dado (dos palabras anteriores más las dos palabras siguientes).
Se crean los vectores a partir de las filas de la matriz resultante.

VECTORIZACION

**k-means**  implementacion del algoritmo

In [21]:
!pip install scikit-learn

In [22]:
import pandas as pd
import re
import spacy
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim.models import Word2Vec

# Leer el archivo XLSX que ya se limpió anteriormente
from google.colab import drive
drive.mount('/gdrive')

# Ruta del archivo en Google Drive
ruta = "/gdrive/My Drive/Limpieza/Data_Completo_Limpio.xlsx"

# Cargo el archivo Excel en un DataFrame de pandas
df = pd.read_excel(ruta)

# Lista de categorías
categorias = df['categoria'].unique()

# Crear un diccionario para mapear categorías a números
categoria_a_numero = {cat: num for num, cat in enumerate(categorias)}

# Iterar sobre cada categoría
for categoria in categorias:
    # Filtrar las filas que pertenecen a la categoría específica
    df_categoria = df[df['categoria'] == categoria]

    # Seleccionar aleatoriamente 100 registros del DataFrame
    df_sample = df_categoria.sample(n=200, random_state=42)

    # Analizar la columna de texto_noticia para la categoría específica
    text = ' '.join(df_sample['texto_noticia'].astype(str))

    # Normalización del texto
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()

    nlp = spacy.load('es_core_news_sm')
    nlp.max_length = len(text) + 100000
    doc = nlp(text)
    filtered_words = [token.lemma_ for token in doc if token.pos_ in ('NOUN', 'ADJ', 'VERB') and token.lemma_.lower() not in nlp.Defaults.stop_words]
    filtered_text = ' '.join(filtered_words)

    # Contar la frecuencia de las palabras utilizando un diccionario
    word_freq = Counter(filtered_words)

    # Guardar las palabras más frecuentes en un archivo de texto con codificación UTF-8
    with open(f'palabras_mas_frecuentes_{categoria}.txt', 'w', encoding='utf-8') as file:
        for word, freq in word_freq.most_common():
            file.write(f"{word}: {freq}\n")

    # Cargar las palabras más frecuentes desde el archivo de texto
    palabras_mas_frecuentes = []
    with open(f'palabras_mas_frecuentes_{categoria}.txt', 'r', encoding='utf-8') as file:
        for line in file:
            palabra = line.split(":")[0].strip()
            palabras_mas_frecuentes.append(palabra)

    # Eliminar duplicados de la lista de palabras más frecuentes
    palabras_mas_frecuentes = list(set(palabras_mas_frecuentes))

    # Vectorización con reducción de dimensionalidad mediante umbral de frecuencia (TF-IDF)
    vectorizer_tfidf = TfidfVectorizer(vocabulary=palabras_mas_frecuentes)
    X_tfidf = vectorizer_tfidf.fit_transform(df_sample['texto_noticia'])
    palabras_tfidf = vectorizer_tfidf.get_feature_names_out()

    # Aplicar K-Means
    num_clusters = 3  # Ajusta según sea necesario
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    df_sample['cluster'] = kmeans.fit_predict(X_tfidf)

    # Imprimir las palabras en cada cluster y su asociación con la categoría
    print(f"Palabras más frecuentes en cada cluster para la categoría {categoria}:")
    for cluster_num in range(num_clusters):
        cluster_words = [palabra for palabra in palabras_mas_frecuentes if kmeans.cluster_centers_[cluster_num][vectorizer_tfidf.vocabulary_[palabra]] > 0]
        print(f"Cluster {cluster_num} - Categoría: {categoria_a_numero[categoria]}")
        print(cluster_words)
# Guardar los resultados de cada cluster en un archivo de texto
    with open(f'resultados_clusters_{categoria}.txt', 'w', encoding='utf-8') as result_file:
        for cluster_num in range(num_clusters):
            cluster_words = [palabra for palabra in palabras_mas_frecuentes if kmeans.cluster_centers_[cluster_num][vectorizer_tfidf.vocabulary_[palabra]] > 0]
            result_file.write(f"Cluster {cluster_num} - Categoría: {categoria_a_numero[categoria]}\n")
            result_file.write(', '.join(cluster_words) + '\n\n')



Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Palabras más frecuentes en cada cluster para la categoría ciencia_tecnologia:
Cluster 0 - Categoría: 0
['empresa', 'tenés', 'visual', 'plan', 'seguir', 'seguimiento', 'contenido', 'años', 'recogido', 'android', 'listado', 'actividad', 'fundado', 'natural', 'ocasión', 'compromiso', 'navegador', 'only', 'defender', 'suscripción', 'miércoles', 'abril', 'huella', 'monopolio', 'demanda', 'neuronal', 'antimonopolio', 'torno', 'thing', 'drive', 'correo', 'ilimitado', 'tiktok', 'comprobar', 'iniciar', 'colección', 'remoto', 'giroscopio', 'información', 'premium', 'diferencia', 'clic', 'diseñadora', 'cp', 'emojis', 'mayo', 'inicio', 'seleccionar', 'rat', 'habitual', 'lunes', 'obtener', 'persona', 'multimillonario', 'utilizado', 'puesto', 'cambio', 'modalidad', 'limitar', 'dedicado', 'tuits', 'obstante', 'lanzar', 'defecto', 'social', 'teléfono', 'determinado', 'reunión', 'prueba', 'derecho', 'intercambiar', 'carácter', 'llamada', 'económico', 'tuit', 'malla', 'solución', 'calendar', 'autenticac

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Palabras más frecuentes en cada cluster para la categoría cultura:
Cluster 0 - Categoría: 1
['filtro', 'surrey', 'combinado', 'empresa', 'zóhar', 'secretario', 'visual', 'arraigado', 'lydia', 'entidad', 'pináculo', 'preparado', 'yam', 'éxito', 'derecha', 'filosofía', 'parroquia', 'testigo', 'talentoso', 'privilegio', 'babilonia', 'seguir', 'levantar', 'contenido', 'capricho', 'reconocida', 'interdependiente', '1584', 'años', 'tronco', 'arriesgar', 'autenticidad', 'computación', 'fijación', 'biotecnología', 'reflejo', 'favor', 'nacionalidad', 'actividad', 'glasgow', 'fundado', 'natural', 'trasatlántico', 'misión', 'apic', 'sistémico', 'blanco', 'paraíso', 'deber', 'convincente', 'jackie', 'venecia', 'elaborado', 'anuncio', 'financiero', 'reina', 'solidaridad', 'ocasión', 'prevención', 'folleto', 'rosalía', 'compromiso', 'democratizar', 'obstrucción', 'valla', 'defender', 'espina', 'frío', 'pensión', 'suelo', 'capucha', 'prematuramente', 'local', 'lanchin', 'xviii', 'contratado', 'desenm

...

Palabras más frecuentes en cada cluster para la categoría deportes:
Cluster 0 - Categoría: 7
['combinado', 'desayuno', 'fino', 'secretario', 'plasmar', 'entidad', 'éxito', 'derecha', 'prematuro', 'privilegio', 'seguir', 'giovannetti', 'años', 'favor', 'nacionalidad', 'actividad', 'rosarino', 'daniela', 'reina', 'compromiso', 'defender', 'zalazar', 'suelo', 'local', 'miércoles', 'exenganche', 'abril', 'recurrir', 'huella', 'arco', 'mónica', 'técnico', 'álvarez', 'empiece', 'retorno', 'cuidar', 'janeiro', 'representante', 'rival', 'camila', 'frentazo', 'correo', 'ex', 'mediocampo', 'sobrecargar', 'issa', 'dilecto', 'amateur', 'pádel', 'paz', 'emilio', 'cronograma', 'básquetbol', 'falcón', 'liga', 'festejo', 'estero', 'ida', 'información', 'ameghino', 'ángel', 'diferencia', 'dominado', 'mayo', 'tomo', 'vigente', 'inicio', 'cemento', 'martina', 'retrasar', 'chocobar', 'habitual', 'lunes', 'jujuy', 'obtener', 'persona', 'pablo', 'puesto', 'descenso', 'predio', 'cambio', 'leal', 'directiva',

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Palabras más frecuentes en cada cluster para la categoría economia:
Cluster 0 - Categoría: 8
['impulsado', 'anticipación', 'resumen', 'desayuno', 'sake', 'empresa', 'secretario', 'tenés', 'visual', 'plan', 'nber', 'entidad', 'evaluó', 'preparado', 'aluminio', 'éxito', 'derecha', 'archipiélago', 'michelini', 'filosofía', 'tomarte', 'prematuro', 'testigo', 'privatización', 'armonizar', 'partimos', 'privilegio', 'seguir', 'seguimiento', 'levantar', 'albany', 'contenido', 'capricho', 'interdependiente', 'invite', 'años', 'multiplicado', 'arriesgar', 'monto', 'descarte', 'consider', 'computación', 'solvente', 'biotecnología', 'reflejo', 'recogido', 'favor', 'nacionalidad', 'reclutamiento', 'contribuir', 'listado', 'rolex', 'actividad', 'glasgow', 'fundado', 'natural', 'compensar', 'inaugural', 'granjero', 'misión', 'sistémico', 'becerra', 'blanco', 'lidiar', 'paraíso', 'deber', 'daniela', 'anticipado', 'elaborado', 'sosa', 'anuncio', 'financiero', 'ocasión', 'compromiso', 'acentuado', 'demo

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Palabras más frecuentes en cada cluster para la categoría salud:
Cluster 0 - Categoría: 9
['surrey', 'resumen', 'hereditario', 'empresa', 'secretario', 'tenés', 'plan', 'conner', 'arraigado', 'entidad', 'dévora', 'éxito', 'espontaneidad', 'archipiélago', 'prematuro', 'testigo', 'privatización', 'armonizar', 'obstetricia', 'privilegio', 'seguir', 'seguimiento', 'levantar', 'fisioterapia', 'contenido', 'años', 'monto', 'biotecnología', 'reflejo', 'recogido', 'favor', 'delatar', 'contribuir', 'actividad', 'fundado', 'natural', 'misión', 'entusiasmar', 'sistémico', 'blanco', 'lidiar', 'amparo', 'financiero', 'reina', 'prevención', 'ocasión', 'irene', 'distanciamiento', 'compromiso', 'cocinar', 'excepción', 'defender', 'pensión', 'suelo', 'local', 'andrea', 'miércoles', 'brote', 'bar', 'abril', 'extraño', 'recurrir', 'senasa', 'horacio', 'génova', 'ecdc', 'potencialidad', 'cumplimentar', 'potasio', 'ruso', 'mónica', 'jaula', 'demanda', 'chandler', 'invertir', 'moneda', 'técnico', 'traumatis